In [3]:
import subprocess

def extract_audio_from_video(video_path, audio_path):
    command = ["ffmpeg", "-i", video_path, "-vn", audio_path]
    subprocess.run(command)


In [4]:
extract_audio_from_video("../files/podcast.mp4", "../files/podcast_audio.mp3")

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex

In [1]:
from pydub import AudioSegment

In [2]:
track = AudioSegment.from_mp3("../files/podcast_audio.mp3")

In [3]:
track.duration_seconds

13413.796281179139

In [10]:
13413.796281179139 / 60

223.56327135298565

In [7]:
five_min_track = track[:5*60*1000]

In [8]:
five_min_track.export("../files/first_five.mp3", format="mp3")

<_io.BufferedRandom name='../files/first_five.mp3'>

In [9]:
len(track) / (10 * 60 * 1000)

22.356326666666668

In [11]:
import math

In [17]:
minutes = 10
min2ms = 60 * 1000
sep_ms = minutes * min2ms
chunck_length = math.ceil(len(track) / sep_ms)
print(chunck_length)

23


In [18]:
for i in range(chunck_length):
    start_part = i * sep_ms
    end_part = (i+1) * sep_ms
    file_name = f"podcast_chunk_{i*minutes:03d}.mp3"

    chunk = track[start_part:end_part]
    chunk.export(f"../files/audio_chunks/{file_name}")

In [1]:
def cut_audio_in_chunks(audio_path, chunk_size, chunk_dir):
    track = AudioSegment.from_mp3(audio_path)
    min2ms = 60 * 1000
    sep_ms = chunk_size * min2ms
    chunck_length = math.ceil(len(track) / sep_ms)

    for i in range(chunck_length):
        start_part = i * sep_ms
        end_part = (i+1) * sep_ms
        file_name = f"podcast_chunk_{i*chunk_size:03d}.mp3"

        chunk = track[start_part:end_part]
        chunk.export(f"{chunk_dir}/{file_name}")
    

In [8]:
import openai
from glob import glob

In [3]:
transciprt = openai.Audio.transcribe("whisper-1", open("../files/sample_chunks/podcast_chunk_000.mp3", "rb"))

In [7]:
print(transciprt["text"])

So I was trying to figure out how to do photorealistic AI photos and it was stable diffusion by itself is not doing that well, like the faces look all mangled. And it doesn't have enough resolution or something to do that well. But I started seeing these base models, these fine-tuned models, and people would train on porn and I would try them and they would be very photorealistic. They would have bodies that actually made sense, like body anatomy. But if you look at the photorealistic models that people use now still, there's still a core of porn there, like of naked people. So I need to prompt out the naked, and everyone needs to do this with AI startups, with imaging, you need to prompt out the naked stuff. You have to keep reminding the model you need to put clothes on. Yeah, don't put naked because it's very risky. I have Google Vision that checks every photo before it's shown to the user to like check. Like a nipple detector? Oh, NSFW detector. Because you get the journalists get 

In [11]:
def transcribe_chunks(chunk_dir, destination):
    files = sorted(glob(f"{chunk_dir}/*.mp3"))
    for file in files:
        with open(file, "rb") as audio_file, open(destination, "a") as text_file:
            transciprt = openai.Audio.transcribe("whisper-1", audio_file)
            text_file.write(transciprt["text"]+"\n")

In [12]:
transcribe_chunks("../files/sample_chunks", "../files/sample_text/sample_transcriptions.txt")